In [1]:
import mpmath as mpm
from mpmath import matrix, mp, mpf
import numpy as np
from tqdm import tqdm

from decimal import Decimal, getcontext

import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from jorbit.utils.doubledouble import DoubleDouble, dd_sum
from jorbit.integrators.mpmath_testing_only import *

mp.dps = 50
getcontext().prec = 50

In [2]:
x = mpf("3.18581409464903452912384824")
y = mpf("9.49294909318055492351329582")

a = DoubleDouble.from_string(str(x))
b = DoubleDouble.from_string(str(y))

result = a * b
print(mpf(float(result.hi)) + mpf(float(result.lo)) - (x * y))

result = a + b
print(mpf(float(result.hi)) + mpf(float(result.lo)) - (x + y))

result = a - b
print(mpf(float(result.hi)) + mpf(float(result.lo)) - (x - y))

result = a / b
print(mpf(float(result.hi)) + mpf(float(result.lo)) - (x / y))

2.1582222975576478539934149278373743961497099320943e-31
6.445565717117830707556082808675453712706815616342e-32
2.5334085155718360282498363906645028262839123763496e-32
6.7485533593586716298401313299413046310542252282098e-34


In [3]:
from jorbit.utils.generate_coefficients import create_iasnn_constants

n_internal_points = 7
h, r, c, d = create_iasnn_constants(n_internal_points)


his = jnp.array([DoubleDouble.from_string(str(x)).hi for x in h])
los = jnp.array([DoubleDouble.from_string(str(x)).lo for x in h])
hq = DoubleDouble(his, los)

_hq = []
for i in range(n_internal_points):
    _hq.append(mpf(float(hq.hi[i])) + mpf(float(hq.lo[i])))
hq = _hq

for i in range(7):
    print(h[i] - hq[i])

100%|██████████| 998/998 [00:00<00:00, 1424.42it/s]


0.0
-1.78829309269048297253238322924787355592787492958996680886023416291806759923e-34
3.70710982590535791694379972333205680911905331799745415126332986895057849701e-35
6.18282342266663273754892956657801107241662437870901946219920168849341061699e-35
2.07281613555046637460758031575514849073333840486369936308026380820683579635e-33
2.54721370516911613738496307575047962553323396933970856560883179735748224712e-33
1.14157532119143081778309690074646968573253814386196532289862758560578809468e-33


In [4]:
h, r, c, d = create_iasnn_constants(n_internal_points)
d = matrix(d)

_, _, _, d_dd = create_iasnn_constants(n_internal_points)
his = jnp.array([DoubleDouble.from_string(str(x)).hi for x in d_dd])
los = jnp.array([DoubleDouble.from_string(str(x)).lo for x in d_dd])
d_dd = DoubleDouble(his, los)

for i in range(7):
    print(d[i] - (mpf(float(d_dd.hi[i])) + mpf(float(d_dd.lo[i]))))

100%|██████████| 998/998 [00:00<00:00, 1439.55it/s]


-1.78829309269048297253238322924787355592787492958996680886023416291806759923e-34
-1.7113551145962831115696532890922173222249534560101169963401143035201572473e-36
6.28613766744899623138590851342504139913643716779470248911474651397026285804e-34
-4.62998438542823728432008279906230982514563825598076430844257827911295041337e-37
4.3116643690931802696463803308196024953902832805368823134960469582727553399e-35
2.23118595648135463295886250107622610546829121004952255896986070164439255988e-33
4.0683195107693548902759990705888412401306290152179433609354714447528017819e-38


In [5]:
from jorbit.integrators.ias15_dd import setup_iasnn_integrator


pre_dd = setup_iasnn_integrator(n_internal_points=7)
b_x_denoms_dd, b_v_denoms_dd, h_dd, r_dd, c_dd, d_dd = pre_dd

b_dd = DoubleDouble(jnp.ones((7, 1, 3), dtype=jnp.float64))
for i in range(7):
    b_dd[i] += DoubleDouble.from_string(str(i))

g_dd = dd_sum((b_dd[None, :, :, :] * d_dd[:, :, None, None]), axis=1)

g_dd

100%|██████████| 998/998 [00:00<00:00, 1397.48it/s]


DoubleDouble([[[ 1.12278765  1.12278765  1.12278765]]

 [[ 2.94595374  2.94595374  2.94595374]]

 [[ 7.46338752  7.46338752  7.46338752]]

 [[18.97006697 18.97006697 18.97006697]]

 [[31.96307196 31.96307196 31.96307196]]

 [[25.29068904 25.29068904 25.29068904]]

 [[ 7.          7.          7.        ]]], [[[-6.88530307e-17 -6.88530307e-17 -6.88530307e-17]]

 [[-2.21781895e-17 -2.21781895e-17 -2.21781895e-17]]

 [[ 4.14115990e-16  4.14115990e-16  4.14115990e-16]]

 [[ 9.01975259e-16  9.01975259e-16  9.01975259e-16]]

 [[ 8.58937064e-16  8.58937064e-16  8.58937064e-16]]

 [[ 6.76468065e-16  6.76468065e-16  6.76468065e-16]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]])

In [6]:
_, _, _, _, _, d_matrix = precompute(7)

b = mpm.ones(7, 3)
for i in range(7):
    b[i, :] = b[i, :] + mpf(str(i))
g = d_matrix * b

100%|██████████| 998/998 [00:00<00:00, 1397.87it/s]


In [7]:
for i in range(7):
    for j in range(3):
        print(g[i, j] - mpf(float(g_dd[i, 0, j].hi)) - mpf(float(g_dd[i, 0, j].lo)))

3.79552477455102955563008858226122482518962319013034267517309798712800552492e-33
3.79552477455102955563008858226122482518962319013034267517309798712800552492e-33
3.79552477455102955563008858226122482518962319013034267517309798712800552492e-33
-9.96991444380999725689424508658344831182022545666683707612310925470629541603e-34
-9.96991444380999725689424508658344831182022545666683707612310925470629541603e-34
-9.96991444380999725689424508658344831182022545666683707612310925470629541603e-34
4.68037696389136908271492228515915589085807563195533396162280640823691475906e-32
4.68037696389136908271492228515915589085807563195533396162280640823691475906e-32
4.68037696389136908271492228515915589085807563195533396162280640823691475906e-32
5.10492484165477455902548711921723339142339373716629356713568897815717671769e-32
5.10492484165477455902548711921723339142339373716629356713568897815717671769e-32
5.10492484165477455902548711921723339142339373716629356713568897815717671769e-32
3.03087176929653777506816

In [8]:
n_internal_points = 7
pre = precompute(7)
b_x_denoms, b_v_denoms, h, r, c, d = pre

dt = mpf("0.01")
x0 = matrix([["1.0", "0", "0"]])
v0 = matrix([["0", "1.0", "0"]])
a0 = matrix([["-1.0", "0", "0"]])
b = mpm.ones(7, 3)  # missing the dimension for nparticles to work w/ mpmath
for i in range(7):
    b[i, :] = b[i, :] + mpf(str(i))

# initialize the gs
g = d * b

for n in [4]:
    x = estimate_x_v_from_b(
        a0=a0,
        v0=v0,
        x0=x0,
        dt=dt,
        b_x_denoms=b_x_denoms,
        b_v_denoms=b_v_denoms,
        h=h[n],
        bp=b,
    )
x

100%|██████████| 998/998 [00:00<00:00, 1293.72it/s]


matrix(
[['0.0000097222222222222222222222222222222222222222222222222', '0.0000097222222222222222222222222222222222222222222222222', '0.0000097222222222222222222222222222222222222222222222222'],
 ['0.000010714285714285714285714285714285714285714285714286', '0.000010714285714285714285714285714285714285714285714286', '0.000010714285714285714285714285714285714285714285714286'],
 ['0.000011904761904761904761904761904761904761904761904762', '0.000011904761904761904761904761904761904761904761904762', '0.000011904761904761904761904761904761904761904761904762'],
 ['0.000013333333333333333333333333333333333333333333333333', '0.000013333333333333333333333333333333333333333333333333', '0.000013333333333333333333333333333333333333333333333333'],
 ['0.000015', '0.000015', '0.000015'],
 ['0.000016666666666666666666666666666666666666666666666667', '0.000016666666666666666666666666666666666666666666666667', '0.000016666666666666666666666666666666666666666666666667'],
 ['0.000016666666666666666666666666

In [14]:
from jorbit.integrators.ias15_dd import _estimate_x_v_from_b as exvfd

pre_dd = setup_iasnn_integrator(n_internal_points=7)
b_x_denoms_dd, b_v_denoms_dd, h_dd, r_dd, c_dd, d_matrix_dd = pre_dd


x0_dd = DoubleDouble(jnp.array([[1.0, 0, 0]]))
v0_dd = DoubleDouble(jnp.array([[0, 1.0, 0]]))
a0_dd = -x0
b_dd = DoubleDouble(jnp.ones((7, 1, 3), dtype=jnp.float64))
for i in range(7):
    b_dd[i] += DoubleDouble.from_string(str(i))


dt_dd = DoubleDouble.from_string("0.01")


a0_dd = -x0_dd


# misc setup, make partialized versions of the helpers that bake in the constants
n_internal_points = b_dd.hi.shape[0]


# initialize the g coefficients
g = dd_sum((b_dd[None, :, :, :] * d_matrix_dd[:, :, None, None]), axis=1)

for n in [4]:
    x_dd = exvfd(
        a0=a0_dd,
        v0=v0_dd,
        x0=x0_dd,
        dt=dt_dd,
        b_x_denoms=b_x_denoms_dd,
        b_v_denoms=b_v_denoms_dd,
        h=h_dd[n],
        bp=b_dd,
    )
x_dd.hi[:, 0, :]

100%|██████████| 998/998 [00:00<00:00, 1403.50it/s]


Array([[ 9.72222222e-06,  9.72222222e-06,  9.72222222e-06],
       [ 1.07142857e-05,  1.07142857e-05,  1.07142857e-05],
       [ 1.19047619e-05,  1.19047619e-05,  1.19047619e-05],
       [ 1.33333333e-05,  1.33333333e-05,  1.33333333e-05],
       [ 1.50000000e-05,  1.50000000e-05,  1.50000000e-05],
       [ 1.66666667e-05,  1.66666667e-05,  1.66666667e-05],
       [ 1.66666667e-05,  1.66666667e-05,  1.66666667e-05],
       [-5.00000000e-05,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e-02,  0.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00]],      dtype=float64)

In [15]:
for i in range(10):
    for j in range(3):
        print(x[i, j] - (mpf(float(x_dd.hi[i, 0, j])) + mpf(float(x_dd.lo[i, 0, j]))))

-1.85955892113014100638592574527407613603667128861400375001567725215388766994e-37
-1.85955892113014100638592574527407613603667128861400375001567725215388766994e-37
-1.85955892113014100638592574527407613603667128861400375001567725215388766994e-37
3.29944471499375786194299353258075215937088040274735993109374251306607034429e-39
3.29944471499375786194299353258075215937088040274735993109374251306607034429e-39
3.29944471499375786194299353258075215937088040274735993109374251306607034429e-39
-7.99246597084807140533957164996033013566930169229624904949626296541695936231e-38
-7.99246597084807140533957164996033013566930169229624904949626296541695936231e-38
-7.99246597084807140533957164996033013566930169229624904949626296541695936231e-38
-5.63666781803098920273197780073856028925916117076011479147510300794978904183e-38
-5.63666781803098920273197780073856028925916117076011479147510300794978904183e-38
-5.63666781803098920273197780073856028925916117076011479147510300794978904183e-38
-6.1208461045056839

In [11]:
import struct


def binary(input: float) -> str:
    return "".join(format(c, "08b") for c in struct.pack("!f", input))


xx = DoubleDouble.from_string("7.0")
yy = DoubleDouble.from_string("0.01")
x2 = DoubleDouble(7.0)
y2 = DoubleDouble(0.01)

print(binary(float(xx.hi)))
print(binary(float(x2.hi)))
print()
print(binary(float(xx.lo)))
print(binary(float(x2.lo)))
print()
print(binary(float(yy.hi)))
print(binary(float(y2.hi)))
print()
print(binary(float(yy.lo)))
print(binary(float(y2.lo)))

01000000111000000000000000000000
01000000111000000000000000000000

00000000000000000000000000000000
00000000000000000000000000000000

00111100001000111101011100001010
00111100001000111101011100001010

10100000011101011100001010001111
10101101001000111101011100001010
